# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
          
    """
    
    # ********************************************************
    #            FLOW
    # ********************************************************
    #
    #     1. Find Slope & Intercept of line segments from Hough
    #     2. segregate lines into left/right based on slope
    #     3. compute avg slope and Intercept
    #     4. project collapsed left/right lines
    # ********************************************************
    
    
    # *************************************************
    # Filter Out Horizontal/vertical Lines
    # *************************************************
    # Lines array is of  Nx1X4 dimensions, each observation 
    # has x1, x2, y1, y2 coordinates.
    #                   x-cord
    lines = lines[ (lines[:,:, 0] != lines[:,:, 2] ) ]



    # ************************************************
    # compute slope, y-intercept and append to lines
    # ************************************************
    ##     slope  =   (     y2      -    y1        ) /  (     x2      -    x1   )
    slope =    ( lines[:,3] - lines[:,1] ) / ( lines[:,2] - lines[:,0] )

    ##       b    =     y         -       m      *     x
    intercept =  lines[:,1] - (slope*lines[:,0])

    ## append to lines array
    intercept.shape = (intercept.shape[0],1)    # typecasting vector from above operation to array
    slope.shape = (slope.shape[0],1)            # typecasting vector from above operation to array
    lines = np.append(lines, slope, 1)
    lines = np.append(lines,intercept,1)



    # ***********************************************
    #    Filter Lines angle < 30deg
    # ***********************************************
    lines = lines[ abs(lines[:,4]) > 0.57]



    # **********************************************
    # Categorize left and Right lines
    # **********************************************
    left_lines  = lines[ lines[:,4] < 0]
    right_lines = lines[ lines[:,4] > 0]


    # left line slope & intercept
    left_line_slope     = np.mean(left_lines[:,4])
    left_line_intercept = np.mean(left_lines[:,5])
    
    # right line slope and intercept
    right_line_slope    =  np.mean(right_lines[:,4])
    right_line_intercept = np.mean(right_lines[:,5])
    

    
    # **********************************************
    # Draw Left & Right Lane Lines    
    # **********************************************
    # Image dimensions
    imht, imwid, imdep  =  img.shape

    # Limit lane line vertical span to 40% of Image Height,
    # compute x by plugging in slope,y and intercept
    y1 = math.floor(0.6 * imht)
    y2 = imht

    cv2.line(img,                            \
             ( math.floor( (y1 - left_line_intercept) / left_line_slope ) , y1 ),  \
             ( math.floor( (y2 - left_line_intercept) / left_line_slope ) , y2 ),  \
             color,     \
             thickness, \
             )

    cv2.line(img,                            \
             ( math.floor( (y1 - right_line_intercept) / right_line_slope ) , y1 ),  \
             ( math.floor( (y2 - right_line_intercept) / right_line_slope ) , y2 ),  \
             color,     \
             thickness, \
             )
    # ***** End of draw_lines *****
    
    
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img
    
# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [3]:
import os
os.listdir("test_images/")

['solidWhiteRight.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowLeft.jpg',
 'solidYellowCurve.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve2.jpg']

run your solution on all test_images and make copies into the test_images directory).

In [4]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.


# Identify Lane Lines
def getEdges(image):

    ## Parameters for Gaussian blurr
    kernal = 3             # Filter size for gaussian smoothing
    gray_image = gaussian_blur(image, kernal)        # Smoothing

    ## Parameters to Extract Edges
    ## to accomodate varying enviroment conditions, will choose 1:3 low to
    ## high threshold.
    low_thld = 100          
    high_thld = 200                                    
    
    edges = canny(gray_image, low_thld, high_thld)     # Canny Transform
    return(edges)


# Mask Non-Lane Lines

def applyMask(image):
  
    # Apply Mask on Canny Transform output
    # will go with trapezoid mask, 10 and 90 % of image width forms base of trapezoid
    #  42 & 58 % forms the top of trapezoid    
    imwid   = image.shape[1]             # Columms
    imht    = image.shape[0]             # Rows
    
    ll = ( math.floor(imwid*0.1) , imht )       # Lower Left Vertex
    lr = ( math.floor(imwid*0.9) , imht )      # Lower Right Vertex
    tl = ( math.floor(imwid*0.42) , math.floor(imht*0.6) )      # Top Left Vertex
    tr = ( math.floor(imwid*0.58) , math.floor(imht*0.6) )      # Top Right Vertex
    vertices = np.array( [[ ll, tl, tr, lr]], dtype=np.int32)   

    fimage = region_of_interest(image, vertices )
    
    return(fimage)

    

## Parameter to Extract Lane Lines
def getLines(image): 
    lines = hough_lines(                       \
                        image,            # Image \
                        2,                     # rho   \
                        np.pi/180,             # theta \
                        40,                    # Threshold \
                        20,                     # min line length \
                        5,                     # max_line_gap \
                        )
    return(lines)


# Draw lines on image
def mergeLines(image, lines):
    lines_image = cv2.addWeighted( lines, 0.95, image, 1, 0)
    return(lines_image)



In [5]:
def process_image(imag):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    imag_copy = imag.copy()
     
    edges = getEdges(imag)
     
    mask_imag = applyMask(edges)
     
    lines = getLines(mask_imag)
     
    line_image = mergeLines(imag_copy, lines)
     
    return(line_image)

In [6]:
import os
os.listdir("test_images/")

['solidWhiteRight.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowLeft.jpg',
 'solidYellowCurve.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve2.jpg']

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [8]:
def process_image(imag):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    imag_copy = imag.copy()
    cv2.imwrite("v_inputImage.jpg",imag)
    
    edges = getEdges(imag)
    cv2.imwrite("v_edges.jpg",edges)
    
    mask_imag = applyMask(edges)
    cv2.imwrite("v_mask.jpg",mask_imag)
    
    lines = getLines(mask_imag)
    cv2.imwrite("v_lines.jpg",lines)
    
    line_image = mergeLines(imag_copy, lines)
    
    cv2.imwrite("videoFrame.jpg", line_image)

    return(line_image)

Let's try the one with the solid white lane on the right first ...

In [9]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:59<00:00,  3.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 14.4 s, sys: 548 ms, total: 14.9 s
Wall time: 1min 1s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [11]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


 31%|███       | 209/682 [00:46<01:42,  4.60it/s]

ValueError: cannot convert float NaN to integer

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## My Views on Lane finding pipeline :
  #### Likely to fail:
      1. Varying enviroment conditions(tree shades, light, rain) affect edge detection.
      2. Fail to Identify edges on concrete roads. 
      3. Single noisy-edge filtering can't fit for far/near sight regions.
      4. Near sight Noise(shapes on road/vehicle) are strong enough to skew projected lane lines.
      5. Straight lane projects may not be nest fit for curvy lane lines.
      6. Curb Spacing could impact Lane line identification.
      7. Short "safe driving distance" could block enough view required for lane line identificaiton.
      8. This pipelines assumes fixed camera position/angle, any change would either need 
            re-tuning of pipeline parameters.
      
  #### Improvements to pipeline:
      1. Need targetted noise filtering for far/near sight regions.
      2. weighted avg method for finding slope and intercept.
      3. should have controls for camera tuning for varying environment conditions.
      4. mix of color selection and edge detection might improve lane finding on concrete roads.
      5. Non-linear fitting model to cover curvy lanes.
      6. Seperate mask for left/right lanes.
  
 



## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [12]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4



 39%|███▉      | 98/251 [00:46<01:34,  1.62it/s]

ValueError: cannot convert float NaN to integer

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))